In [1]:
import pandas as pd
import random


In [2]:
df = pd.read_excel("accounts.xlsx")
df.head()

,_airbyte_ab_id,_airbyte_emitted_at,cin,fax,pan,ssn,url,zip,city,flg1,...,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,_airbyte_additional_properties,source_file_path,updated_at
0,273b5728-535b-4106-8c31-ce5e48eee381,2024-08-05 07:02:37.777,NaN,NaN,NaN,NaN,NaN,977000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
1,70ced3d3-ad57-4433-a382-6ecbcc42fd90,2024-08-05 07:02:37.777,NaN,NaN,NaN,NaN,NaN,0,KATHM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
2,15e20dca-a337-44b5-b202-b299895336df,2024-08-05 07:02:37.777,NaN,NaN,NaN,NaN,NaN,977000,BUDAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
3,5cf04c55-8085-4e86-ab6f-88904ceb2c6e,2024-08-05 07:02:37.777,NaN,NaN,NaN,NaN,NaN,977000,RAMNA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596
4,8fb8b8e6-aa95-4344-bdbe-a49935add064,2024-08-05 07:02:37.777,NaN,NaN,104894561.0,NaN,NaN,12,BIRAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s3a://ai360nica/data/bronze/oracle/cbs/CRMUSER...,2024-08-07 06:29:18.596


In [3]:
df = df.drop(['_airbyte_ab_id', '_airbyte_emitted_at',"source_file_path","_airbyte_additional_properties"], axis=1)
df.head()

,cin,fax,pan,ssn,url,zip,city,flg1,flg2,flg3,...,sencitizenconversionflag,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at
0,NaN,NaN,NaN,NaN,NaN,977000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596
1,NaN,NaN,NaN,NaN,NaN,0,KATHM,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596
2,NaN,NaN,NaN,NaN,NaN,977000,BUDAN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596
3,NaN,NaN,NaN,NaN,NaN,977000,RAMNA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596
4,NaN,NaN,104894561.0,NaN,NaN,12,BIRAT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596


In [4]:
#cust_type generation
import numpy as np
def get_cust_type(df):
    df= df.copy()
    condition =  (df["constitution_code"] == "INDIJ") | (df["constitution_code"] == "INDIV")
    df['cust_type'] = np.where(condition,"INDIVIDUAL","LEGAL")
    return df



df = get_cust_type(df)
df[["cust_type","constitution_code"]].head(7)

,cust_type,constitution_code
0,INDIVIDUAL,INDIV
1,LEGAL,OTHER
2,INDIVIDUAL,INDIV
3,INDIVIDUAL,INDIJ
4,INDIVIDUAL,INDIV
5,INDIVIDUAL,INDIV
6,LEGAL,USTS


In [5]:
# CIF_ID Generation
import pandas as pd
import numpy as np

def generate_cif_id(code):
    if code == "INDIVIDUAL":
        prefix = "DLNIND"
    elif code == "LEGAL":
        prefix = "DLNLEG"
    else:
        return ""  
    
    random_digits = str(np.random.randint(1000000, 9999999))
    return prefix + random_digits


df["cif_id"] = df["cust_type"].apply(generate_cif_id)
df["core_cust_id"] = df["cif_id"]
df["group_id"] = df["cif_id"]
df["introducerid"] = ""

df.head()

,cin,fax,pan,ssn,url,zip,city,flg1,flg2,flg3,...,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,cif_id,group_id
0,NaN,NaN,NaN,NaN,NaN,977000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND7968999,DLNIND7968999
1,NaN,NaN,NaN,NaN,NaN,0,KATHM,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG3737602,DLNLEG3737602
2,NaN,NaN,NaN,NaN,NaN,977000,BUDAN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2014270,DLNIND2014270
3,NaN,NaN,NaN,NaN,NaN,977000,RAMNA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND3825870,DLNIND3825870
4,NaN,NaN,104894561.0,NaN,NaN,12,BIRAT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND8597606,DLNIND8597606


In [6]:
df["cif_id"].nunique()

125

In [7]:
import numpy as np
import random


df["marital_status"] = np.where(
    df["cust_type"] == "INDIVIDUAL",
    np.random.choice([True, False],size=len(df), p=[0.62, 0.38]),
    ""
)
df[["cust_type","marital_status"]].head(5)

,cust_type,marital_status
0,INDIVIDUAL,True
1,LEGAL,
2,INDIVIDUAL,True
3,INDIVIDUAL,True
4,INDIVIDUAL,False


In [8]:
selected_columns = [
    "cif_id",
    "orgkey",
    "cust_first_name",
    "cust_middle_name",
    "cust_last_name",
    "cust_dob",
    "cust_type",
    "gender",
    "marital_status",
    "education",
    "pan",
    "email",
    "preferredphone"
]
remaining_columns = [col for col in df.columns if col not in selected_columns]


In [9]:
new_df = df[selected_columns + remaining_columns]
new_df.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND7968999,319749297,SHYAM,NaN,CHAUDHARY,1988-02-17T00:00:00.000000,INDIVIDUAL,F,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND7968999
1,DLNLEG3737602,R24249553,NaN,NaN,NIC BANK-LOAN PROCESSING FEE,1900-01-01T00:00:00.000000,LEGAL,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG3737602
2,DLNIND2014270,R28893337,PRADYUMNA,LAL,RAJBAMSHI,1961-12-06T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2014270
3,DLNIND3825870,R66368828,CHANDESHWAR,SAHUTELI/SHAMBHUKUMAR,SAH,1965-07-16T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND3825870
4,DLNIND8597606,R02063811,JAGADISH,PRASAD,AGRAWAL,1943-04-05T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND8597606


In [10]:
cust_Ind = new_df[new_df["cust_type"]=="INDIVIDUAL"]
cust_Ind.head(80)

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND7968999,319749297,SHYAM,NaN,CHAUDHARY,1988-02-17T00:00:00.000000,INDIVIDUAL,F,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND7968999
2,DLNIND2014270,R28893337,PRADYUMNA,LAL,RAJBAMSHI,1961-12-06T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2014270
3,DLNIND3825870,R66368828,CHANDESHWAR,SAHUTELI/SHAMBHUKUMAR,SAH,1965-07-16T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND3825870
4,DLNIND8597606,R02063811,JAGADISH,PRASAD,AGRAWAL,1943-04-05T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND8597606
5,DLNIND5109497,R00061514,BIRENDRA BAHADUR,NaN,NEUPANE,1978-06-13T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5109497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,DLNIND7443775,R01911511,TIKA,NaN,KUMARI PADHYA,1959-01-27T00:00:00.000000,INDIVIDUAL,F,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND7443775
121,DLNIND2598658,R01911516,BHARATI,NaN,KUMARI KHATRI,1997-09-15T00:00:00.000000,INDIVIDUAL,F,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2598658
122,DLNIND9139167,R01911519,BASANTI,NaN,DAMAI,1968-04-22T00:00:00.000000,INDIVIDUAL,F,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND9139167
123,DLNIND5073099,R01911512,BUDAI,NaN,SAH,1971-05-05T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5073099


In [11]:
print(len(cust_Ind))

71


In [12]:
df = pd.read_csv("shuffled_data.csv")
df = df.iloc[0:71]

In [13]:
print(len(df))

71


In [15]:
cust_Ind["cust_first_name"] = df["cust_first_name"].values
cust_Ind["name"] = df["cust_first_name"].astype(str) + " " + df["cust_middle_name"].astype(str) + " " + df["cust_last_name"].astype(str)
cust_Ind["preferredname"] =  df["cust_first_name"].astype(str) + " " + df["cust_middle_name"].astype(str) + " " + df["cust_last_name"].astype(str)
cust_Ind["cust_middle_name"] = df["cust_middle_name"].values
cust_Ind["cust_last_name"] = df["cust_last_name"].values
cust_Ind["gender"] = df["gender"].values
cust_Ind["short_name"] = df["cust_first_name"].values
cust_Ind.head(80)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\1309904559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_Ind["cust_first_name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\1309904559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_Ind["name"] = df["cust_first_name"].astype(str) + " " + df["cust_middle_name"].astype(str) + " " + df["cust_last_name"].astype(str)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\1309904559.py:3: SettingWithCopyWarni

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND7968999,319749297,Buddhiraj,Bishal,Sharma,1988-02-17T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND7968999
2,DLNIND2014270,R28893337,Pramil,Krishna,Neupane,1961-12-06T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2014270
3,DLNIND3825870,R66368828,Shirish,Gopal,Thapa,1965-07-16T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND3825870
4,DLNIND8597606,R02063811,Hina,Devi,Bhattarai,1943-04-05T00:00:00.000000,INDIVIDUAL,F,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND8597606
5,DLNIND5109497,R00061514,Poshan,Bahadur,Thapa,1978-06-13T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5109497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,DLNIND7443775,R01911511,Heli,Saraswati,Timalsina,1959-01-27T00:00:00.000000,INDIVIDUAL,F,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND7443775
121,DLNIND2598658,R01911516,Pranij,Bibek,Shrestha,1997-09-15T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2598658
122,DLNIND9139167,R01911519,Sanjita,Kumar,Pandey,1968-04-22T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND9139167
123,DLNIND5073099,R01911512,Prajina,Kumari,Pandey,1971-05-05T00:00:00.000000,INDIVIDUAL,F,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5073099


In [16]:
df = pd.read_csv("company_name.csv")
df = df.iloc[0:54]
df.head()

,cust_first_name,cust_type
0,TANAHUN DIGITAL PVT. LTD.,LEGAL
1,PYUTHAN LOGISTICS PVT. LTD.,LEGAL
2,PATHIVARA LOGISTICS PVT. LTD.,LEGAL
3,DLYTICA ACADEMY,LEGAL
4,PARSA TRADING PVT. LTD.,LEGAL


In [17]:
print(len(df))

54


In [18]:
cust_leg = new_df[new_df["cust_type"]=="LEGAL"]
cust_leg.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
1,DLNLEG3737602,R24249553,NaN,NaN,NIC BANK-LOAN PROCESSING FEE,1900-01-01T00:00:00.000000,LEGAL,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG3737602
6,DLNLEG2271414,C00004434,NaN,NaN,BARPIPAL TOLE BATO UPAVAOKTA SAMITI,2019-01-22T00:00:00.000000,LEGAL,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG2271414
7,DLNLEG6060578,R08272460,NaN,NaN,CRYSTAL OFFSET PRESS,1900-01-01T00:00:00.000000,LEGAL,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG6060578
8,DLNLEG6439748,C00004439,NaN,NaN,ARJUN SUNCHANDI UDHYOG,2015-09-15T00:00:00.000000,LEGAL,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG6439748
9,DLNLEG5255359,C00004426,NaN,NaN,ROYAL FURNITURE UDYOG,2019-01-31T00:00:00.000000,LEGAL,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG5255359


In [19]:
print(len(cust_leg))

54


In [21]:
cust_leg["cust_first_name"] = df["cust_first_name"].values
cust_leg["name"] = df["cust_first_name"].values
cust_leg["preferredname"] = df["cust_first_name"]
cust_leg["cust_middle_name"] = ""
cust_leg["cust_last_name"] = ""
cust_leg["gender"] = ""
cust_leg["short_name"] = df["cust_first_name"].values
cust_leg.head(5)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\3459596799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_leg["cust_first_name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\3459596799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_leg["name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\3459596799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
1,DLNLEG3737602,R24249553,TANAHUN DIGITAL PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG3737602
6,DLNLEG2271414,C00004434,PYUTHAN LOGISTICS PVT. LTD.,,,2019-01-22T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG2271414
7,DLNLEG6060578,R08272460,PATHIVARA LOGISTICS PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG6060578
8,DLNLEG6439748,C00004439,DLYTICA ACADEMY,,,2015-09-15T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG6439748
9,DLNLEG5255359,C00004426,PARSA TRADING PVT. LTD.,,,2019-01-31T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG5255359


In [23]:
result = pd.concat([cust_Ind,cust_leg])
shuffled = result.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND2049255,R01516157,Suprim,Shankar,Acharya,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2049255
1,DLNIND6472052,R01007045,Prithivi,Kumar,Yadav,1966-04-13T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND6472052
2,DLNIND2692249,R09396490,Nawraj,Hari,Regmi,2013-01-09T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2692249
3,DLNLEG3033786,R22132679,MANANG HOSPITALITY PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG3033786
4,DLNIND5653388,R00580783,Shrikumar,Kumar,Sharma,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5653388


In [24]:
import numpy as np

def generate_email(data):
    # Extract first name (handle NaN and company names)
    first_name = str(data["cust_first_name"]) if not pd.isna(data["cust_first_name"]) else "unknown"
    
    # Remove "PVT. LTD." or "LTD." and clean the name
    name = (
        first_name.replace("PVT. LTD.", "")
                 .replace("LTD.", "")
                 .strip() 
                 .lower()  
                 .replace(" ", "")
    )
    
    num = str(np.random.randint(100, 999))
    
    if data["cust_type"] == "INDIVIDUAL":
        email = f"{name}{num}@gmail.com"
    else:
        email = f"{name}{num}@outlook.com"
    
    return email

# Apply the function to the DataFrame
shuffled["email"] = shuffled[["cust_first_name", "cust_type"]].apply(generate_email, axis=1)

# Display first 15 generated emails
shuffled["email"].head(15)

0                   suprim655@gmail.com
1                 prithivi185@gmail.com
2                   nawraj691@gmail.com
3      mananghospitality737@outlook.com
4                shrikumar598@gmail.com
5       tamorelectronics903@outlook.com
6                   hansha861@gmail.com
7          nepalairlines895@outlook.com
8     sarlahiengineering621@outlook.com
9                   poshan267@gmail.com
10                  aavaas335@gmail.com
11        bardiabuilders194@outlook.com
12                  sakhil428@gmail.com
13          khotangfoods209@outlook.com
14               anushikha295@gmail.com
Name: email, dtype: object

In [25]:
shuffled.head(99)

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND2049255,R01516157,Suprim,Shankar,Acharya,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2049255
1,DLNIND6472052,R01007045,Prithivi,Kumar,Yadav,1966-04-13T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND6472052
2,DLNIND2692249,R09396490,Nawraj,Hari,Regmi,2013-01-09T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2692249
3,DLNLEG3033786,R22132679,MANANG HOSPITALITY PVT. LTD.,,,1900-01-01T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG3033786
4,DLNIND5653388,R00580783,Shrikumar,Kumar,Sharma,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5653388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,DLNIND5029687,R01786441,Keith,Bhakta,Rai,2001-08-19T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5029687
95,DLNLEG5255359,C00004426,PARSA TRADING PVT. LTD.,,,2019-01-31T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG5255359
96,DLNIND2379710,R01009724,Mikma,Biren,Thapa,1982-06-01T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2379710
97,DLNLEG6099206,C00012403,PHEWA AUTO PARTS PVT. LTD.,,,2014-04-09T00:00:00.000000,LEGAL,,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNLEG6099206


In [26]:
def generate_phone_number(data):
   return  "+977-98"+str(np.random.randint(10000000,99999999))

shuffled["preferredphone"] = shuffled["preferredphone"].apply(generate_phone_number)
shuffled["preferredphone"].head(10)

0    +977-9896333226
1    +977-9895441296
2    +977-9834332317
3    +977-9887081810
4    +977-9898630656
5    +977-9816249615
6    +977-9846934301
7    +977-9866795366
8    +977-9882425864
9    +977-9815516076
Name: preferredphone, dtype: object

In [27]:
Ind_cust = shuffled[shuffled["cust_type"] == "INDIVIDUAL"]
print(len(Ind_cust))

71


In [28]:
first_5 = Ind_cust.iloc[:5]        
remaining = Ind_cust.iloc[5:] 

In [29]:
first_5.head()

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND2049255,R01516157,Suprim,Shankar,Acharya,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2049255
1,DLNIND6472052,R01007045,Prithivi,Kumar,Yadav,1966-04-13T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND6472052
2,DLNIND2692249,R09396490,Nawraj,Hari,Regmi,2013-01-09T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2692249
4,DLNIND5653388,R00580783,Shrikumar,Kumar,Sharma,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5653388
6,DLNIND9698578,019898447,Hansha,Bibek,Adhikari,1991-06-03T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND9698578


In [31]:
import pandas as pd
import numpy as np

# Create sample data
data = {
    'cust_first_name': ['Prashanta', 'Kabita', 'sandal', 'sirish', 'Veshraj'],
    'cust_middle_name': ['Babu', '', '', '', ''],
    'cust_last_name': ['Thapa', 'Bhattarai', 'Tandukar', 'poudel', 'Mainali'],
    'preferredphone': ['9810288411', '9866180896', '9861396794', '9861632577', '9861674997'],
    'email': ['Prashant.babu@dlytica.com', 'kabita.bhattarai@dlytica.com', 'sandil.tandukar@dlytica.com', 'sirish.poudel@dlytica.com', 'Veshraj.Mainali@dlytica.com'],
    'preferredemail': ['john.smith@example.com', 'emily.johnson@example.com', 'michael.williams@example.com', 'sarah.brown@example.com', 'david.jones@example.com']
}

# Create DataFrame
df = pd.DataFrame(data)

df.head()

,cust_first_name,cust_middle_name,cust_last_name,preferredphone,email,preferredemail
0,Prashanta,Babu,Thapa,9810288411,Prashant.babu@dlytica.com,john.smith@example.com
1,Kabita,,Bhattarai,9866180896,kabita.bhattarai@dlytica.com,emily.johnson@example.com
2,sandal,,Tandukar,9861396794,sandil.tandukar@dlytica.com,michael.williams@example.com
3,sirish,,poudel,9861632577,sirish.poudel@dlytica.com,sarah.brown@example.com
4,Veshraj,,Mainali,9861674997,Veshraj.Mainali@dlytica.com,david.jones@example.com


In [32]:
first_5["cust_first_name"] = df["cust_first_name"].values
first_5["cust_middle_name"] = df["cust_middle_name"].values
first_5["cust_last_name"] = df["cust_last_name"].values
first_5["preferredphone"] = df["preferredphone"].values
first_5["email"] = df["email"].values
first_5["preferredemail"] = df["email"].values
first_5["name"] = df["cust_first_name"].values + " " + df["cust_middle_name"].values + " " + df["cust_last_name"].values
first_5["preferredname"] = df["cust_first_name"].values + " " + df["cust_middle_name"].values + " " + df["cust_last_name"].values
first_5.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\103564471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_5["cust_first_name"] = df["cust_first_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\103564471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_5["cust_middle_name"] = df["cust_middle_name"].values
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22020\103564471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,cif_id,orgkey,cust_first_name,cust_middle_name,cust_last_name,cust_dob,cust_type,gender,marital_status,education,...,creditbureauscorevalidity,foreignacctaxreportingreq,foreigntaxreportingstatus,preferred_mobile_alert_no,credithistoryrequestedflag,foreigntaxreportingcountry,customer_level_provisioning,preferred_mobile_alert_type,updated_at,group_id
0,DLNIND2049255,R01516157,Prashanta,Babu,Thapa,1966-05-06T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2049255
1,DLNIND6472052,R01007045,Kabita,,Bhattarai,1966-04-13T00:00:00.000000,INDIVIDUAL,M,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND6472052
2,DLNIND2692249,R09396490,sandal,,Tandukar,2013-01-09T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND2692249
4,DLNIND5653388,R00580783,sirish,,poudel,2000-01-01T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND5653388
6,DLNIND9698578,019898447,Veshraj,,Mainali,1991-06-03T00:00:00.000000,INDIVIDUAL,M,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-07 06:29:18.596,DLNIND9698578


In [33]:
leg_cust = shuffled[shuffled["cust_type"] == "LEGAL"]
print(len(Ind_cust))

71


In [34]:
result = pd.concat([first_5,remaining,leg_cust])
shuffled = result.sample(frac=1, random_state=42).reset_index(drop=True)

In [35]:
shuffled.to_csv('account_masked.csv', index=False) 